In [21]:
import cv2 as cv
import numpy as np
import imageio
import glob

# Função para processar cada frame do vídeo
def process_frame(frame, background_remover, path):
    result = frame.copy()
    image = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # Definir o intervalo de cores para detecção do vermelho
    lower_red = np.array([165, 30, 20])
    upper_red = np.array([189, 255, 255])

    # Criar máscara para a área vermelha
    mask = cv.inRange(image, lower_red, upper_red)
    red_area = cv.bitwise_and(result, result, mask=mask)

    # Aplicar desfoque para reduzir ruídos
    frame = cv.GaussianBlur(frame, (5, 5), 0)

    # Remover o fundo e aplicar operações morfológicas para obter contornos
    mask = background_remover.apply(red_area)
    _, mask = cv.threshold(mask, 254, 255, cv.THRESH_BINARY)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Encontrar contornos
    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Encontrar o maior contorno válido e desenhar uma elipse sobre ele
    max_cnt = [0, 0]
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 800 and len(cnt) >= 5:
            if area > max_cnt[0]:
                max_cnt = [area, cnt]

    if max_cnt[0] != 0:
        elipse = cv.fitEllipse(max_cnt[1])
        path.append([int(elipse[0][0]), int(elipse[0][1])])
        cv.ellipse(frame, elipse, (0, 255, 0), 2)

    return frame, path

# Função principal para seguir o robô Husky no vídeo
def follow_husky(input_video, name, outpath):
    cap = cv.VideoCapture(input_video)
    background_remover = cv.createBackgroundSubtractorKNN(detectShadows=True)

    frames = []
    path = []

    # Processamento de cada frame do vídeo
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        # Processar o frame atual
        frame, path = process_frame(frame, background_remover, path)
        frames.append(frame)

    cap.release()

    # Salvar o vídeo com a identificação da caixa vermelha
    save_video(frames, outpath, name)

    # Gerar a animação da trajetória do marcador
    generate_path_animation(path, outpath, name)

# Função para salvar o vídeo identificado com a caixa vermelha
def save_video(frames, outpath, name):
    if frames:
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        out = cv.VideoWriter(outpath + name + '.avi', fourcc, 20.0, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()



# Chamada para seguir o robô Husky nos vídeos fornecidos e gerar o GIF da trajetória
follow_husky('C:\\Users\\mauri\\Documents\\GitHub\\ENGG64-Visao_Computacional\\TRABALHO\\Video1_husky.mp4', 'husky1', 'C:\\Users\\mauri\\Documents\\GitHub\\ENGG64-Visao_Computacional\\TRABALHO\\test_v2\\')
follow_husky('C:\\Users\\mauri\\Documents\\GitHub\\ENGG64-Visao_Computacional\\TRABALHO\\video2_husky.mp4', 'husky2', 'C:\\Users\\mauri\\Documents\\GitHub\\ENGG64-Visao_Computacional\\TRABALHO\\test_v2\\')



C:\Users\mauri\AppData\Local\Temp\ipykernel_24200\936798294.py:101: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 316 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  color = np.array([[[hue, saturation, value]]], dtype=np.uint8)
C:\Users\mauri\AppData\Local\Temp\ipykernel_24200\936798294.py:101: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 1824 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  color = np.array([[[hue, saturation, value]]], dtype=np.uint8)
C:\Users\mauri\AppData\Local\Temp\ipykernel_24200\936798294.py:101: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays